<a href="https://colab.research.google.com/github/HimanshuAgarwal022/IDRiDSegmentation/blob/master/IDRiDseg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diabetic Retinopathy Lesion Segmentation.
---

MIT License

Copyright (c) [2020] [Himanshu Agarwal]

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


---



### load drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### convert ground truths to binary

In [0]:
from PIL import Image
#from resizeimage import resizeimage
import os, sys

def cmp(a, b):
    return (a > b) - (a < b) 

def resizeImage(infile,file, output_dir, size=(4288,2848)):
  outfile = os.path.splitext(file)[0]
  extension = os.path.splitext(file)[1]
  #print(outfile)
  #print(extension)
  #print(infile)
  #if (cmp(extension, ".jpg")):
    #print("dsd")
    #return

  if infile != outfile:
    try :
      im = Image.open(infile)
      gray = im.convert('L')
      bw = gray.point(lambda x: 0 if x<50 else 255, '1')
      # im = resizeimage.resize_cover(im, [960, 640])
      bw.save(output_dir+outfile[:-3]+extension,"TIFF",quality=100)
      #print("sucess")
    #except IOError:
    #  print ("cannot reduce image for ", infile)
    except e:
      print (e)
    


output_dir = "drive/My Drive/data/output/annotations/"
annot_dir = "drive/My Drive/data/annotations/"
dir = os.getcwd()

if not os.path.exists(os.path.join(dir,output_dir)):
  os.mkdir(output_dir)
annot = os.path.join(dir,annot_dir)
for file in os.listdir(annot):
  #print(file)
  resizeImage(os.path.join(annot,file),file,output_dir)

### CLAHE on input images

In [0]:
import os, sys
import numpy as np
import cv2
from matplotlib import pyplot as plt

dir = os.getcwd()
dir_data = os.path.join(dir,"drive/My Drive/data/output/preimages/")
output_dir_data = os.path.join(dir,"drive/My Drive/data/output/images/")
if not os.path.exists(os.path.join(dir,output_dir_data)):
    os.mkdir(output_dir_data)

gridsize = 8
for file in os.listdir(dir_data):
  bgr = cv2.imread(os.path.join(dir_data,file))
  lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
  lab_planes = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(gridsize,gridsize))
  lab_planes[0] = clahe.apply(lab_planes[0])
  lab = cv2.merge(lab_planes)
  bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
  #if random.uniform(0, 1.0) > 0.75:
  #  bgr = cv2.bitwise_not(bgr)
  #  plt.imshow(bgr)
  #  plt.show()
  cv2.imwrite(os.path.join(output_dir_data,file),bgr)

In [0]:
#output_dir_data = os.path.join(dir,"drive/My Drive/data/output/images/")
#for file in os.listdir(output_dir_data):
#  bgr = cv2.imread(os.path.join(output_dir_data,file))
#  rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
#  plt.imshow(rgb)
#  plt.show()
#  bgr = cv2.bitwise_not(bgr)
#  plt.imshow(bgr)
#  plt.show()

### extract patches

In [0]:
from PIL import Image
#from resizeimage import resizeimage
import os, sys
import numpy as np
import cv2
from matplotlib import pyplot as plt

dir = os.getcwd()
output_dir_data = "drive/My Drive/data/output/patches/"
output_dir_mask = "drive/My Drive/data/output/labels/"
if not os.path.exists(os.path.join(dir,output_dir_data)):
    os.mkdir(output_dir_data)
if not os.path.exists(os.path.join(dir,output_dir_mask)):
    os.mkdir(output_dir_mask)

dir_data = os.path.join(dir,"drive/My Drive/data/output/images/")
dir_mask = os.path.join(dir,"drive/My Drive/data/output/annotations/")

# im = Image.open(os.path.join(dir_mask,"IDRiD_06.tif"))
# im_crop = im.crop((2000,0,2000+512,0+256))
# im_crop.show()
# image_np = np.array(im_crop)
# print np.sum(image_np)

negative_patches = []
positive_count = 0

for file in os.listdir(dir_mask):
    outfile = os.path.splitext(file)[0]
    extension = os.path.splitext(file)[1]
    #if (cmp(extension, ".jpg")):
    #    continue
    img = outfile + ".jpg"
    im = Image.open(os.path.join(dir_mask,file))
    imd = Image.open(os.path.join(dir_data,img))
    # image_np = np.array(im)
    # print np.sum([True, True])
    # im_crop = im.crop((1900,0,1900+512,0+512))
    patch_id = 0
    for i in range(10): #10 6
    	for j in range(16): #16 9
        top_y = i*256 #256 512
        if (i==9): #9 5
          top_y = 2336
        top_x = j*256 #256 512
        if (j==15): #15 8
          top_x = 3776

        im_crop = im.crop((top_x,top_y,top_x+512,top_y+512))
        imd_crop = imd.crop((top_x,top_y,top_x+512,top_y+512))
        im_crop.save(output_dir_mask+outfile+"_p"+str(patch_id)+extension,"JPEG",quality=100)
        imd_crop.save(output_dir_data+outfile+"_p"+str(patch_id)+extension,"JPEG",quality=100)
        if (np.sum(np.array(im_crop)) < 100):
          negative_patches.append(output_dir_mask+outfile+"_p"+str(patch_id)+extension)
        else:
          positive_count += 1

        patch_id += 1

negative_patches = np.array(negative_patches)
# np.savetxt("negative.csv", negative_patches, delimiter=",", fmt="%s")

negative_count = negative_patches.size
delete_count = negative_count - 4*positive_count
np.random.shuffle(negative_patches)
split_idx = delete_count
delete_patches = negative_patches[:split_idx]

for idx in range(delete_patches.size):
    os.remove(delete_patches[idx])
    os.remove(os.path.join(output_dir_data,delete_patches[idx][34:]))

### split in train and test sets

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

dir = "drive/My Drive/data/output/patches/"
#dir = "drive/My Drive/data/output/images/"
image_paths = os.listdir(dir)

length = len(image_paths)
img_paths = np.empty(length, dtype=object)

i=0
for file in image_paths:
	img_paths[i] = "drive/My Drive/data/output/patches/" + file
	#img_paths[i] = "drive/My Drive/data/output/images/" + file
	#print (img_paths[i])
	i+=1

# print (img_paths)
np.random.shuffle(img_paths)
#split_idx = int(img_paths.shape[0] * 1)
split_idx = 11534
train_paths = img_paths[:split_idx]
test_paths = img_paths[split_idx:]

train_paths_ = np.copy(train_paths)
test_paths_ = np.copy(test_paths)
print("train set: ",train_paths.size)
print("test set: ",test_paths.size)
for i in range(train_paths.size):
	train_paths_[i] = train_paths[i][35:]
	#train_paths_[i] = train_paths[i][34:]
	train_paths_[i] = "drive/My Drive/data/output/labels/" + train_paths_[i]
	#train_paths_[i] = "drive/My Drive/data/output/annotations/" + train_paths_[i]
	#print (train_paths_[i])

#print ("split")

for i in range(test_paths.size):
	test_paths_[i] = test_paths[i][35:]
	#test_paths_[i] = test_paths[i][34:]
	test_paths_[i] = "drive/My Drive/data/output/labels/" + test_paths_[i]
	#test_paths_[i] = "drive/My Drive/data/output/annotations/" + test_paths_[i]
	#print (test_paths_[i])

train_csv = np.stack((train_paths,train_paths_), axis=1)
test_csv = np.stack((test_paths,test_paths_), axis=1)

np.savetxt("train.csv", train_csv, delimiter=",", fmt="%s")
np.savetxt("test.csv", test_csv, delimiter=",", fmt="%s")

train set:  11534
test set:  8


### load tensorflow

In [4]:
#import os
#os.getcwd()
#os.listdir()
#os.path.exists('drive/My Drive/')
#for roots,dirs,files in os.walk('drive/My Drive'):               
#                    print(roots,dirs,files)
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('TensorFlow Version: {}'.format(tf.__version__))
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 1.x selected.
TensorFlow Version: 1.15.2
Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=ee2b3d234e82bb1c13f028526f9f680f5971df1f7db91a5bc6c48c72b186221a
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.4 GB  | Proc size: 557.2 MB
GPU RAM Free: 16015MB | Used: 265MB | Util   2% | Total 16280MB


### Train

In [0]:
#!rm 'drive/My Drive/data/models' -rf
#!rm 'drive/My Drive/data/logs' -rf

In [0]:
import time
import os
import pandas as pd
import tensorflow as tf


def image_augmentation(image, mask):
    """Returns (maybe) augmented images
    (1) Random flip (left <--> right)
    (2) Random flip (up <--> down)
    (3) Random brightness
    (4) Random hue
    Args:
        image (3-D Tensor): Image tensor of (H, W, C)
        mask (3-D Tensor): Mask image tensor of (H, W, 1)
    Returns:
        image: Maybe augmented image (same shape as input `image`)
        mask: Maybe augmented mask (same shape as input `mask`)
    """
    concat_image = tf.concat([image, mask], axis=-1)

    maybe_flipped = tf.image.random_flip_left_right(concat_image)
    maybe_flipped = tf.image.random_flip_up_down(concat_image)

    image = maybe_flipped[:, :, :-1]
    mask = maybe_flipped[:, :, -1:]

    image = tf.image.random_brightness(image, 0.7)
    image = tf.image.random_hue(image, 0.3)

    return image, mask


def get_image_mask(queue, augmentation=True):
    """Returns `image` and `mask`
    Input pipeline:
        Queue -> CSV -> FileRead -> Decode JPEG
    (1) Queue contains a CSV filename
    (2) Text Reader opens the CSV
        CSV file contains two columns
        ["path/to/image.jpg", "path/to/mask.jpg"]
    (3) File Reader opens both files
    (4) Decode JPEG to tensors
    Notes:
        height, width = 640, 960
    Returns
        image (3-D Tensor): (640, 960, 3)
        mask (3-D Tensor): (640, 960, 1)
    """
    text_reader = tf.TextLineReader(skip_header_lines=1)
    _, csv_content = text_reader.read(queue)

    image_path, mask_path = tf.decode_csv(
        csv_content, record_defaults=[[""], [""]])

    image_file = tf.read_file(image_path)
    mask_file = tf.read_file(mask_path)

    image = tf.image.decode_jpeg(image_file, channels=3)
    image.set_shape([512, 512, 3])
    image = tf.cast(image, tf.float32)

    mask = tf.image.decode_jpeg(mask_file, channels=1)
    mask.set_shape([512, 512, 1])
    mask = tf.cast(mask, tf.float32)
    mask = mask / (tf.reduce_max(mask) + 1e-7)

    if augmentation:
        image, mask = image_augmentation(image, mask)

    return image, mask


def conv_conv_pool(input_,
                   n_filters,
                   training,
                   flags,
                   name,
                   pool=True,
                   activation=tf.nn.relu):
    """{Conv -> BN -> RELU}x2 -> {Pool, optional}
    Args:
        input_ (4-D Tensor): (batch_size, H, W, C)
        n_filters (list): number of filters [int, int]
        training (1-D Tensor): Boolean Tensor
        name (str): name postfix
        pool (bool): If True, MaxPool2D
        activation: Activaion functions
    Returns:
        net: output of the Convolution operations
        pool (optional): output of the max pooling operations
    """
    net = input_

    with tf.variable_scope("layer{}".format(name)):
        for i, F in enumerate(n_filters):
            net = tf.layers.conv2d(
                net,
                F, (3, 3),
                activation=None,
                padding='same',
                kernel_regularizer=tf.contrib.layers.l2_regularizer(flags.reg),
                name="conv_{}".format(i + 1))
            net = tf.layers.batch_normalization(
                net, training=training, name="bn_{}".format(i + 1))
            net = activation(net, name="relu{}_{}".format(name, i + 1))

        if pool is False:
            return net

        pool = tf.layers.max_pooling2d(
            net, (2, 2), strides=(2, 2), name="pool_{}".format(name))

        return net, pool


def upconv_concat(inputA, input_B, n_filter, flags, name):
    """Upsample `inputA` and concat with `input_B`
    Args:
        input_A (4-D Tensor): (N, H, W, C)
        input_B (4-D Tensor): (N, 2*H, 2*H, C2)
        name (str): name of the concat operation
    Returns:
        output (4-D Tensor): (N, 2*H, 2*W, C + C2)
    """
    up_conv = upconv_2D(inputA, n_filter, flags, name)

    return tf.concat(
        [up_conv, input_B], axis=-1, name="concat_{}".format(name))


def upconv_2D(tensor, n_filter, flags, name):
    """Up Convolution `tensor` by 2 times
    Args:
        tensor (4-D Tensor): (N, H, W, C)
        n_filter (int): Filter Size
        name (str): name of upsampling operations
    Returns:
        output (4-D Tensor): (N, 2 * H, 2 * W, C)
    """

    return tf.layers.conv2d_transpose(
        tensor,
        filters=n_filter,
        kernel_size=2,
        strides=2,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(flags.reg),
        name="upsample_{}".format(name))


def make_unet(X, training, flags=None):
    """Build a U-Net architecture
    Args:
        X (4-D Tensor): (N, H, W, C)
        training (1-D Tensor): Boolean Tensor is required for batchnormalization layers
    Returns:
        output (4-D Tensor): (N, H, W, C)
            Same shape as the `input` tensor
    Notes:
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        https://arxiv.org/abs/1505.04597
    """
    net = X / 127.5 - 1
    conv1, pool1 = conv_conv_pool(net, [16, 16], training, flags, name=1)
    conv2, pool2 = conv_conv_pool(pool1, [32, 32], training, flags, name=2)
    conv3, pool3 = conv_conv_pool(pool2, [64, 64], training, flags, name=3)
    conv4, pool4 = conv_conv_pool(pool3, [128, 128], training, flags, name=4)
    conv5, pool5 = conv_conv_pool(pool4, [256, 256], training, flags, name=5)
    conv6, pool6 = conv_conv_pool(pool5, [512, 512], training, flags, name=6)
    conv7 = conv_conv_pool(
        pool6, [1024, 1024], training, flags, name=7, pool=False)

    up8 = upconv_concat(conv7, conv6, 512, flags, name=8)
    conv8 = conv_conv_pool(up8, [512, 512], training, flags, name=8, pool=False)

    up9 = upconv_concat(conv8, conv5, 256, flags, name=9)
    conv9 = conv_conv_pool(up9, [256, 256], training, flags, name=9, pool=False)

    up10 = upconv_concat(conv9, conv4, 128, flags, name=10)
    conv10 = conv_conv_pool(up10, [128, 128], training, flags, name=10, pool=False)

    up11 = upconv_concat(conv10, conv3, 64, flags, name=11)
    conv11 = conv_conv_pool(up11, [64, 64], training, flags, name=11, pool=False)

    up12 = upconv_concat(conv11, conv2, 32, flags, name=12)
    conv12 = conv_conv_pool(up12, [32, 32], training, flags, name=12, pool=False)

    up13 = upconv_concat(conv12, conv1, 16, flags, name=13)
    conv13 = conv_conv_pool(up13, [16, 16], training, flags, name=13, pool=False)

    # return tf.layers.conv2d(
    #     conv13,
    #     1, (1, 1),
    #     name='final',
    #     activation=tf.nn.sigmoid,
    #     padding='same')
    return tf.layers.conv2d(conv13,1, (1, 1),name='final',activation=None,padding='same')

def BCE_(y_pred, y_true):
    # weight ratio = 9:1
    # 9-1=8
    class_weights = tf.constant([8],dtype=tf.float32)
    tensor_one = tf.constant([1],dtype=tf.float32)

    pred_flat = tf.reshape(y_pred, [-1, 1])
    true_flat = tf.reshape(y_true, [-1, 1])

    weight_map = tf.multiply(true_flat, class_weights)
    weight_map = tf.add(weight_map, tensor_one)

    loss_map = tf.nn.sigmoid_cross_entropy_with_logits(logits=pred_flat, labels=true_flat)
    loss_map = tf.multiply(loss_map, weight_map)
    loss = tf.reduce_mean(loss_map)
    return loss

def IOU_(y_pred, y_true):
    """Returns a (approx) IOU score
    intesection = y_pred.flatten() * y_true.flatten()
    Then, IOU = 2 * intersection / (y_pred.sum() + y_true.sum() + 1e-7) + 1e-7
    Args:
        y_pred (4-D array): (N, H, W, 1)
        y_true (4-D array): (N, H, W, 1)
    Returns:
        float: IOU score
    """
    H, W, _ = y_pred.get_shape().as_list()[1:]
    threshold = 0.7
    pred_flat = tf.reshape(y_pred, [-1, H * W])
    true_flat = tf.reshape(y_true, [-1, H * W])
    pred = tf.cast(pred_flat > threshold, dtype=tf.float32)
    true = tf.cast(true_flat > threshold, dtype=tf.float32)
    intersection = tf.reduce_sum(pred * true, axis=1) + 1e-7
    denominator = tf.reduce_sum(pred, axis=1) + tf.reduce_sum(true, axis=1) + 1e-7

    return tf.reduce_mean(intersection / denominator)


def make_train_op(y_pred, y_true):
    """Returns a training operation
    Args:
        y_pred (4-D Tensor): (N, H, W, 1)
        y_true (4-D Tensor): (N, H, W, 1)
    Returns:
        train_op: minimize operation
    """
    # loss = -IOU_(y_pred, y_true)
    loss = BCE_(y_pred, y_true)

    global_step = tf.train.get_or_create_global_step()

    # optim = tf.train.AdamOptimizer()
    optim = tf.train.AdamOptimizer(1e-4)
    return optim.minimize(loss, global_step=global_step)
class flags:
  epochs = 100
  batch_size = 8
  logdir = "drive/My Drive/data/logs/"
  reg = 0.1
  ckdir = "drive/My Drive/data/models/"


'''def read_flags():
    """Returns flags"""

    import argparse

    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument(
        "--epochs", default=1, type=int, help="Number of epochs")

    parser.add_argument("--batch-size", default=8, type=int, help="Batch size")

    parser.add_argument(
        "--logdir", default="logdir", help="Tensorboard log directory")

    parser.add_argument(
        "--reg", type=float, default=0.1, help="L2 Regularizer Term")

    parser.add_argument(
        "--ckdir", default="models", help="Checkpoint directory")

    flags = parser.parse_args()
    return flags'''


def main():
    train = pd.read_csv("./train.csv")
    n_train = train.shape[0]

    test = pd.read_csv("./test.csv")
    n_test = test.shape[0]

    current_time = time.strftime("%m/%d/%H/%M/%S")
    train_logdir = os.path.join(flags.logdir, "train", current_time)
    test_logdir = os.path.join(flags.logdir, "test", current_time)

    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=[None, 512, 512, 3], name="X")
    y = tf.placeholder(tf.float32, shape=[None, 512, 512, 1], name="y")
    mode = tf.placeholder(tf.bool, name="mode")

    pred = make_unet(X, mode, flags)

    tf.add_to_collection("inputs", X)
    tf.add_to_collection("inputs", mode)
    tf.add_to_collection("outputs", pred)

    tf.summary.histogram("Predicted Mask", pred)
    tf.summary.image("Predicted Mask", pred)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.control_dependencies(update_ops):
        train_op = make_train_op(pred, y)

    IOU_op = IOU_(pred, y)
    IOU_op = tf.Print(IOU_op, [IOU_op])
    tf.summary.scalar("IOU", IOU_op)

    train_csv = tf.train.string_input_producer(['train.csv'])
    test_csv = tf.train.string_input_producer(['test.csv'])
    train_image, train_mask = get_image_mask(train_csv)
    test_image, test_mask = get_image_mask(test_csv, augmentation=False)

    X_batch_op, y_batch_op = tf.train.shuffle_batch(
        [train_image, train_mask],
        batch_size=flags.batch_size,
        capacity=flags.batch_size * 5,
        min_after_dequeue=flags.batch_size * 2,
        allow_smaller_final_batch=True)

    X_test_op, y_test_op = tf.train.batch(
        [test_image, test_mask],
        batch_size=flags.batch_size,
        capacity=flags.batch_size * 2,
        allow_smaller_final_batch=True)

    summary_op = tf.summary.merge_all()

    with tf.Session() as sess:
        train_summary_writer = tf.summary.FileWriter(train_logdir, sess.graph)
        test_summary_writer = tf.summary.FileWriter(test_logdir)

        init = tf.global_variables_initializer()
        sess.run(init)

        saver = tf.train.Saver()
        if os.path.exists(flags.ckdir):
            latest_check_point = tf.train.latest_checkpoint(flags.ckdir)
            saver.restore(sess, latest_check_point)
            print('model restored!')

        else:
            #try:
            #    os.rmdir(flags.ckdir)
            #except IOError:
            #    pass
            os.mkdir(flags.ckdir)

        try:
            global_step = tf.train.get_global_step(sess.graph)

            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)
            start = time.time()
            for epoch in range(90,flags.epochs):
                print('%d epochs in %fs' % (epoch, 62539+time.time()-start))
                for step in range(0, n_train, flags.batch_size):
                    #print("%d train steps in %fs" %(step, time.time()-start))
                    X_batch, y_batch = sess.run([X_batch_op, y_batch_op])

                    _, step_iou, step_summary, global_step_value = sess.run(
                        [train_op, IOU_op, summary_op, global_step],
                        feed_dict={X: X_batch,
                                   y: y_batch,
                                   mode: True})

                    train_summary_writer.add_summary(step_summary,
                                                     global_step_value)

                total_iou = 0
                for step in range(0, n_test, flags.batch_size):
                    #print("%d test steps in %fs" %(step, time.time()-start))
                    X_test, y_test = sess.run([X_test_op, y_test_op])
                    step_iou, step_summary = sess.run(
                        [IOU_op, summary_op],
                        feed_dict={X: X_test,
                                   y: y_test,
                                   mode: False})

                    total_iou += step_iou * X_test.shape[0]

                    test_summary_writer.add_summary(step_summary,(epoch + 1) * (step + 1))

                #saver.save(sess, "{}/model.ckpt".format(flags.ckdir))
                #print('the %d epoch in %fs, saved successfully' % (epoch, time.time()-start))

        finally:
            coord.request_stop()
            coord.join(threads)
            saver.save(sess, "{}/model.ckpt".format(flags.ckdir))
            print('model saved successfully')



if __name__ == '__main__':
    #flags = read_flags()
    main()

INFO:tensorflow:Summary name Predicted Mask is illegal; using Predicted_Mask instead.
INFO:tensorflow:Summary name Predicted Mask is illegal; using Predicted_Mask instead.
INFO:tensorflow:Restoring parameters from drive/My Drive/data/models/model.ckpt
model restored!
90 epochs in 62539.000005s
91 epochs in 63012.843219s
92 epochs in 63480.010369s
93 epochs in 63947.371985s
94 epochs in 64414.004312s
95 epochs in 64883.230457s
96 epochs in 65350.314995s
97 epochs in 65820.104939s
98 epochs in 66288.175114s
99 epochs in 66757.795235s
model saved successfully


In [0]:
%load_ext tensorboard
%tensorboard --logdir 'drive/My Drive/data/logs'
#%reload_ext tensorboard

### Test on a single image

In [7]:
import tensorflow as tf
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.measurements import label
from PIL import Image
from scipy.special import expit

saver = tf.train.import_meta_graph("drive/My Drive/data/models/model.ckpt.meta")
sess = tf.InteractiveSession()
saver.restore(sess, "drive/My Drive/data/models/model.ckpt")
X, mode = tf.get_collection("inputs")[:2]
pred = tf.get_collection("outputs")[0]

def read_image(image_path, gray=False):
    """Returns an image array
    Args:
        image_path (str): Path to image.jpg
    Returns:
        3-D array: RGB numpy image array
    """
    if gray:
        return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    image = cv2.imread(image_path)    
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def pipeline(image, threshold=0.5, image_WH=(512, 512)):
    image = np.copy(image)
    H, W, C = image.shape
    
    if (W, H) != image_WH:
        image = cv2.resize(image, image_WH)
    
    mask_pred = sess.run(pred, feed_dict={X: np.expand_dims(image, 0),
                                          mode: False})
    
    mask_pred = np.squeeze(mask_pred)
    mask_pred = expit(mask_pred)
    # mask_pred = mask_pred > threshold
    return mask_pred

image_path = "drive/My Drive/data/output/images/IDRiD_17.jpg"
image = read_image(image_path)
predicted_image = np.zeros((2848, 4288), dtype=float)

for i in range(10): #10 6
  for j in range(16): #16 9
    top_y = i*256 #256 512
    if (i==9): #9 5
      top_y = 2336
    top_x = j*256 #256 512
    if (j==15): #15 8
      top_x = 3776

    image_crop = image[top_y:top_y+512, top_x:top_x+512]
    predicted_crop = pipeline(image_crop)
    predicted_image[top_y:top_y+512, top_x:top_x+512] = np.maximum(predicted_image[top_y:top_y+512, top_x:top_x+512], predicted_crop)

threshold = 0.7
predicted_image = predicted_image > threshold
(unique, counts) = np.unique(predicted_image.astype('uint8')*255, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

predicted_save = Image.fromarray((predicted_image.astype('uint8'))*255)
predicted_save.save("test_predicted.jpg", "JPEG")

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from drive/My Drive/data/models/model.ckpt
[[       0 10224722]
 [     255  1987502]]


### Generate probability maps for the dataset using the trained model

In [8]:
import tensorflow as tf
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.measurements import label
from PIL import Image
from scipy.special import expit
import os, sys

saver = tf.train.import_meta_graph("drive/My Drive/data/models/model.ckpt.meta")
sess = tf.InteractiveSession()
saver.restore(sess, "drive/My Drive/data/models/model.ckpt")
X, mode = tf.get_collection("inputs")[:2]
pred = tf.get_collection("outputs")[0]

def read_image(image_path, gray=False):
    """Returns an image array
    Args:
        image_path (str): Path to image.jpg
    Returns:
        3-D array: RGB numpy image array
    """
    if gray:
        return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    image = cv2.imread(image_path)    
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def pipeline(image, image_WH=(512, 512)):
    image = np.copy(image)
    H, W, C = image.shape
    
    if (W, H) != image_WH:
        image = cv2.resize(image, image_WH)
    
    mask_pred = sess.run(pred, feed_dict={X: np.expand_dims(image, 0),
                                          mode: False})
    
    mask_pred = np.squeeze(mask_pred)
    mask_pred = expit(mask_pred)
    # mask_pred = mask_pred > threshold
    return mask_pred

output_dir = "drive/My Drive/data/output/prob/"
dir = os.getcwd()

if not os.path.exists(os.path.join(dir,output_dir)):
    os.mkdir(output_dir)

for image_path in os.listdir(os.path.join(dir,"drive/My Drive/data/output/images/")):
    image = read_image("drive/My Drive/data/output/images/"+image_path)
    predicted_image = np.zeros((2848, 4288), dtype=float)

    for i in range(10): #10 6
      for j in range(16): #16 9
        top_y = i*256 #256 512
        if (i==9): #9 5
          top_y = 2336
        top_x = j*256 #256 512
        if (j==15): #15 8
          top_x = 3776

        image_crop = image[top_y:top_y+512, top_x:top_x+512]
        predicted_crop = pipeline(image_crop)
        predicted_image[top_y:top_y+512, top_x:top_x+512] = np.maximum(predicted_image[top_y:top_y+512, top_x:top_x+512], predicted_crop)

    # threshold = 0.5
    # predicted_image = predicted_image > threshold
    #(unique, counts) = np.unique((predicted_image*255).astype('uint8'), return_counts=True)
    #frequencies = np.asarray((unique, counts)).T
    #print(frequencies)
    predicted_save = Image.fromarray((predicted_image*255).astype('uint8'))
    predicted_save.save(output_dir+image_path, "JPEG", quality=100)

INFO:tensorflow:Restoring parameters from drive/My Drive/data/models/model.ckpt


/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### plot FROC curve

In [9]:
from PIL import Image
import numpy as np
import os, sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

training_size=81
gt_dir = "drive/My Drive/data/output/annotations/"
prob_dir = "drive/My Drive/data/output/prob/"
true_p=0
actual_p=0
pred_p=0
false_p=0

thresh_list = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999, 0.9999, 0.99999, 1]

dir = os.getcwd()
thresh_size = len(thresh_list)
sn = np.empty(thresh_size, dtype=float)
fppi = np.empty(thresh_size, dtype=float)
thresh_array = np.array(thresh_list)

for th in range(thresh_size):
	threshold = thresh_array[th]
	print (threshold)
	true_p=0
	actual_p=0
	pred_p=0
	false_p=0

	for image_path in os.listdir(os.path.join(dir,gt_dir)):
		# print image_path
		im_gt = Image.open(gt_dir+image_path)
		img = os.path.splitext(image_path)[0] + ".jpg"
		im_prob = Image.open(prob_dir+img)
		arr_gt = np.array(im_gt)
		#(unique, counts) = np.unique(arr_gt, return_counts=True)
		#frequencies = np.asarray((unique, counts)).T
		#print("arr_gt: ",frequencies)
		arr_prob = (np.array(im_prob)).astype(float)/255
		#(unique, counts) = np.unique(arr_prob, return_counts=True)
		#frequencies = np.asarray((unique, counts)).T
		#print("arr_prob: ",frequencies)
		arr_pred = (arr_prob > threshold).astype('uint8')
		#(unique, counts) = np.unique(arr_pred, return_counts=True)
		#frequencies = np.asarray((unique, counts)).T
		#print("arr_pred: ",frequencies)
		tp = np.sum(np.logical_and(arr_gt, arr_pred))
		ap = np.sum(arr_gt)
		pp = np.sum(arr_pred)
		true_p += tp
		actual_p += ap
		pred_p += pp
		false_p += (pp-tp)

	sn[th] = float(true_p)/float(actual_p)
	print ("sn: ", sn[th])
	fppi[th] = float(false_p)/float(training_size)
	print ("fppi: ", fppi[th])

plt.plot(fppi, sn)
plt.ylabel('SN')
plt.xlabel('FPs per image')
plt.savefig('drive/My Drive/data/froc.png')

0.0
sn:  0.9997918539315748
fppi:  433847.8024691358
1e-05
sn:  0.9997918539315748
fppi:  433847.8024691358
0.0001
sn:  0.9997918539315748
fppi:  433847.8024691358
0.001
sn:  0.9997918539315748
fppi:  433847.8024691358
0.01
sn:  0.9994911652226554
fppi:  298734.4567901235
0.1
sn:  0.9943516045316544
fppi:  163824.34567901236
0.2
sn:  0.9918498585006056
fppi:  133057.64197530865
0.3
sn:  0.9889898217071691
fppi:  113391.08641975309
0.4
sn:  0.985346916103857
fppi:  97763.20987654322
0.5
sn:  0.9808762180787789
fppi:  85151.83950617284
0.6
sn:  0.9739881305827571
fppi:  72394.50617283951
0.7
sn:  0.9626368323303055
fppi:  59131.567901234564
0.8
sn:  0.9414782104509493
fppi:  44431.61728395062
0.9
sn:  0.892350447299412
fppi:  26864.46913580247
0.99
sn:  0.5611847614316616
fppi:  2681.543209876543
0.999
sn:  0.001640410645737987
fppi:  13.061728395061728
0.9999
sn:  0.001640410645737987
fppi:  13.061728395061728
0.99999
sn:  0.001640410645737987
fppi:  13.061728395061728
1.0
sn:  0.0
fppi

### Generate segmented output masks from the probability maps

In [15]:
import tensorflow as tf
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.measurements import label
from PIL import Image
from scipy.special import expit
import os, sys

saver = tf.train.import_meta_graph("drive/My Drive/data/models/model.ckpt.meta")
sess = tf.InteractiveSession()
saver.restore(sess, "drive/My Drive/data/models/model.ckpt")
X, mode = tf.get_collection("inputs")[:2]
pred = tf.get_collection("outputs")[0]

def read_image(image_path, gray=False):
    """Returns an image array
    Args:
        image_path (str): Path to image.jpg
    Returns:
        3-D array: RGB numpy image array
    """
    if gray:
        return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    image = cv2.imread(image_path)    
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def pipeline(image, image_WH=(512, 512)):
    image = np.copy(image)
    H, W, C = image.shape
    
    if (W, H) != image_WH:
        image = cv2.resize(image, image_WH)
    
    mask_pred = sess.run(pred, feed_dict={X: np.expand_dims(image, 0),
                                          mode: False})
    
    mask_pred = np.squeeze(mask_pred)
    mask_pred = expit(mask_pred)
    # mask_pred = mask_pred > threshold
    return mask_pred

output_dir = "drive/My Drive/data/output/predicted/"
dir = os.getcwd()

if not os.path.exists(os.path.join(dir,output_dir)):
    os.mkdir(output_dir)

for image_path in os.listdir(os.path.join(dir,"drive/My Drive/data/output/prob/")):
    im_prob = Image.open("drive/My Drive/data/output/prob/"+image_path)
    arr_prob = (np.array(im_prob)).astype(float)/255
    threshold = 0.8
    arr_pred = (arr_prob > threshold).astype('uint8')
    # image = read_image("test_data/"+image_path)
    # predicted_image = np.zeros((2848, 4288), dtype=float)

    # for i in range(10):
    #     for j in range(16):
    #         top_y = i*256
    #         if (i==9):
    #             top_y = 2336
    #         top_x = j*256
    #         if (j==15):
    #             top_x = 3776

    #         image_crop = image[top_y:top_y+512, top_x:top_x+512]
    #         predicted_crop = pipeline(image_crop)
    #         predicted_image[top_y:top_y+512, top_x:top_x+512] = np.maximum(predicted_image[top_y:top_y+512, top_x:top_x+512], predicted_crop)
    #(unique, counts) = np.unique(arr_pred*255, return_counts=True)
    #frequencies = np.asarray((unique, counts)).T
    #print(frequencies)
    predicted_save = Image.fromarray(arr_pred*255)
    predicted_save.save(output_dir+image_path, "JPEG", quality=100)

INFO:tensorflow:Restoring parameters from drive/My Drive/data/models/model.ckpt


/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### Calculate sensitivity and precison values for individual images

In [0]:
from PIL import Image
import numpy as np
import os, sys

training_size=80
gt_dir = "drive/My Drive/data/output/annotations/"
pred_dir = "drive/My Drive/data/output/predicted/"
sn = np.empty(training_size, dtype=float)
ppv = np.empty(training_size, dtype=float)
sp = np.empty(training_size, dtype=float)
image_paths = np.empty(training_size, dtype=object)

dir = os.getcwd()
i=0
for image_path in os.listdir(os.path.join(dir,gt_dir)):
	image_paths[i] = image_path
	im_gt = Image.open(gt_dir+image_path)
	img = os.path.splitext(image_path)[0] + ".jpg"
	im_pred = Image.open(pred_dir+img)
	arr_gt = np.array(im_gt)
	arr_pred = np.array(im_pred)
	arr_pred = arr_pred > 0
	#(unique, counts) = np.unique(arr_gt, return_counts=True)
	#frequencies = np.asarray((unique, counts)).T
	#print("arr_gt: ",frequencies)
	#(unique, counts) = np.unique(arr_pred, return_counts=True)
	#frequencies = np.asarray((unique, counts)).T
	#print("arr_pred: ",frequencies)
	true_p = np.sum(np.logical_and(arr_gt, arr_pred))
	actual_p = np.sum(arr_gt)
	pred_p = np.sum(arr_pred)
	
	false_p = pred_p - true_p
	actual_n = 4288*2848 - actual_p
	true_n = actual_n - false_p
	#print ("True pos: ", true_p)
	#print ("Actual pos: ", actual_p)
	#print ("Pred pos: ", pred_p)
	if actual_p == 0:
		sn[i] = 1
	else:
		sn[i] = float(true_p)/float(actual_p)
	if pred_p == 0:
		ppv[i] = 1
	else:
		ppv[i] = float(true_p)/float(pred_p)
	#print (i)
	if actual_n == 0:
		sp[i] = 1
	else:
		sp[i] = float(true_n)/float(actual_n)
	i+=1

sn_csv = np.stack((image_paths,sn), axis=1)
ppv_csv = np.stack((image_paths,ppv), axis=1)
sp_csv = np.stack((image_paths,sp), axis=1)

np.savetxt("drive/My Drive/data/sn.csv", sn_csv, delimiter=",", fmt="%s")
np.savetxt("drive/My Drive/data/ppv.csv", ppv_csv, delimiter=",", fmt="%s")
np.savetxt("drive/My Drive/data/sp.csv", sp_csv, delimiter=",", fmt="%s")

### compute average statistics

In [17]:
from PIL import Image
import numpy as np
import os, sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#np.set_printoptions(threshold=sys.maxsize)

gt_dir = "drive/My Drive/data/output/annotations/"
pred_dir = "drive/My Drive/data/output/predicted/"
true_p=0
actual_p=0
pred_p=0
false_p=0
false_n=0
actual_n=0
true_n=0
pred_n=0

dir = os.getcwd()

for image_path in os.listdir(os.path.join(dir,gt_dir)):
  im_gt = Image.open(gt_dir+image_path)
  img = os.path.splitext(image_path)[0] + ".jpg"
  im_pred = Image.open(pred_dir+img)
  arr_gt = np.array(im_gt)
  arr_pred = np.array(im_pred)
  arr_pred = arr_pred > 0
  #(unique, counts) = np.unique(arr_pred, return_counts=True)
  #frequencies = np.asarray((unique, counts)).T
  #print("arr_pred: ",frequencies)

  tp = np.sum(np.logical_and(arr_gt, arr_pred))
  #print("tp: ",tp)
  ap = np.sum(arr_gt)
  #print("ap: ",ap)
  pp = np.sum(arr_pred)

  fp = pp - tp
  an = 4288*2848 - ap
  pn = 4288*2848 - pp
  tn = an - fp
  fn = pn - tn

  true_p += tp
  actual_p += ap
  pred_p += pp
  pred_n += pn
  false_p += fp
  actual_n += an
  true_n += tn
  false_n +=fn


sn = float(true_p)/float(actual_p)
ppv = float(true_p)/float(pred_p)
sp = float(true_n)/float(actual_n)
npv = float(true_n)/float(pred_n)
acc = float(true_p + true_n)/float(actual_p + actual_n)
f1 = float(2*true_p)/float((2*true_p)+false_p+false_n)

print ("Sensitivity/Recall/True Positive Rate(TPR)(TP/P): ", sn)
print ("Precision/Positive Predictive Value(PPV)(TP/TP+FP): ", ppv)
print ("Specificity/Selectivity/True Negative Rate(TNR)(TN/N): ", sp)
print ("Negative Predictive Value(NPV)(TN/TN+FN): ", npv)
print ("Accuracy(TP+TN/P+N)",acc)
print ("F1 Score(2TP/2TP+FP+FN)",f1)

Sensitivity/Recall/True Positive Rate(TPR)(TP/P):  0.9425759438650518
Precision/Positive Predictive Value(PPV)(TP/TP+FP):  0.7188802057557396
Specificity/Selectivity/True Negative Rate(TNR)(TN/N):  0.9961816109225247
Negative Predictive Value(NPV)(TN/TN+FN):  0.9994032052285645
Accuracy(TP+TN/P+N) 0.9956319893084176
F1 Score(2TP/2TP+FP+FN) 0.8156690606860414


### precision score

In [18]:
from PIL import Image
import numpy as np
import os, sys
from sklearn.metrics import average_precision_score

training_size=80
gt_dir = "drive/My Drive/data/output/annotations/"
prob_dir = "drive/My Drive/data/output/prob/"

dir = os.getcwd()

i=0
sum_pav=0
for image_path in os.listdir(os.path.join(dir,gt_dir)):
	# print image_path
	im_gt = Image.open(gt_dir+image_path)
	img = os.path.splitext(image_path)[0] + ".jpg"
	im_prob = Image.open(prob_dir+img)
	arr_gt = (np.array(im_gt)).astype(bool)
	arr_prob = (np.array(im_prob)).astype(float)/255
	#(unique, counts) = np.unique(arr_gt, return_counts=True)
	#frequencies = np.asarray((unique, counts)).T
	#print("arr_prob: ",frequencies)
	pav = average_precision_score(arr_gt.reshape((-1)),arr_prob.reshape((-1)))
	sum_pav = sum_pav+pav
	i = i+1

mpav = sum_pav/i
print(mpav)

0.7908225819055167
